In [1]:

import pandas as pd
import numpy as np
import sys

from numpy.ma.extras import average
from python310.Lib.test.test_audioop import datas

print('Python version:', sys.version)
print('Pandas version:', pd.__version__)
print('Numpy version:', np.__version__)

Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Pandas version: 2.2.3
Numpy version: 2.1.2


In [60]:
data_s = pd.read_excel('./data/湖南.xls',index_col=0)

In [3]:
import sys
print(sys.executable)

D:\Develop\python\python.exe


In [61]:
data_n = pd.read_excel('./data/湖北.xls')

In [5]:
# 判断有无重复行
data_n.duplicated().value_counts()
data_s.duplicated().value_counts()

False    13361
Name: count, dtype: int64

In [62]:
# 缺失值检测,如果某个字段的缺失值数量超过一千,则删除该字段
miss_count = data_s.isna().sum()
data_s = data_s.loc[:,miss_count<=1000]

In [ ]:
# 另外一种处理缺失值的方法
# thresh：设置一个整数阈值，指定非缺失值的最小数量。如果一行或一列中的非缺失值数量小于这个阈值，就会被删除。
data_s.dropna(axis=0,thresh = data_s.shape[0] - 1000,inplace=True)

In [63]:
miss_count2 = data_n.isnull().sum()
data_n = data_n.loc[:,miss_count2<=1000]

In [ ]:
# 运用sweetviz快速统计每一列信息
import sweetviz as sv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
report = sv.analyze(data_s)
report.show_html('湖南2.html')

In [64]:
columns = ['发货流水号','送货客户','地址流水号','应收客户','开票客户','送货人','商务签收人名','物流签收人名',
'财务签收时间','财务签收人','放货人','销售业务员','原始单据扫描信息','创建人NO','创建人名称','提交人NO',
 '提交人名称','核算部门节点NO','成本中心','币种','是否是费用发货'
]
columns2 =['发货流水号','送货客户','地址流水号','应收客户','开票客户','送货人','商务签收人名','物流签收人名',
'财务签收时间','财务签收人','放货人','销售业务员','原始单据扫描信息','创建人NO','创建人名称','提交人NO',
 '提交人名称','核算部门节点NO','成本中心','币种','打印人', '打印时间'
]

In [65]:
data_s.drop(columns,axis=1,inplace=True)
data_n.drop(columns2,axis=1,inplace=True)

In [66]:
data_s['province'] = '湖南'
data_n['province'] = '湖北'

In [67]:
data = pd.concat([data_s,data_n[data_s.columns]],axis=0)

In [68]:
# select_dtypes用于选择某种特定类型的数据
data_num = data.select_dtypes(['int','float'])
data_cat = data.select_dtypes(['object'])
data_time = data.select_dtypes(['datetime'])

In [69]:
# 数值形数据
# 确保数据可以分组
data_num['province'] = data['province'].map({"湖南":1,'湖北':2})
data_num = data_num.groupby("province").transform(lambda x:x.fillna(x.mean()))

In [70]:
data_cat = data_cat.groupby('province').transform(lambda x:x.ffill())

In [71]:
data_cat['province'] = data['province']

In [72]:
##时间空数据的处理
data_time[['商务签收时间','物流签收时间']] = data_time[['商务签收时间','物流签收时间']].ffill(axis = 1)
data_time.drop('商务签收时间',axis=1,inplace=True)

In [80]:
data = pd.concat([data_num,data_cat,data_time],axis=1)

In [81]:
data = data.set_index('发货时间').loc['2020-01'].reset_index()

In [83]:
data.to_csv('./data/湖南湖北.xlsx',index=None)

In [82]:
data

,发货时间,发货明细数量,打印次数,回款金额,发货小包装数量,拒收总数量,发货金额,拣货小包装数量,发货净重,发货毛重,...,是否门店仓飞单,是否采购退转采购入,是否同步单据到售后,快递运输是否报价,province,送货时间,物流签收时间,放货时间,创建时间,提交时间
0,2020-01-01 00:00:00,0,0,0.0,80,0,30598.00,80,1.503400,1.800000,...,N,N,N,N,湖南,2020-01-02 02:37:33,2020-01-02 02:37:37,2020-01-02 02:37:31,2020-01-02 02:35:01,2020-01-02 02:37:31
1,2020-01-01 00:00:00,0,0,0.0,67,0,30067.00,67,1.094000,1.279800,...,N,N,N,N,湖南,2020-01-02 02:39:23,2020-01-02 02:39:26,2020-01-02 02:39:21,2020-01-02 02:35:01,2020-01-02 02:39:21
2,2020-01-01 00:00:00,0,0,0.0,75,0,42911.00,75,1.084800,1.270500,...,N,N,N,N,湖南,2020-01-02 02:39:54,2020-01-02 02:39:57,2020-01-02 02:39:53,2020-01-02 02:35:01,2020-01-02 02:39:53
3,2020-01-01 00:00:00,0,0,0.0,29,0,13414.00,29,0.455600,0.542900,...,N,N,N,N,湖南,2020-01-02 02:40:30,2020-01-02 02:40:32,2020-01-02 02:40:29,2020-01-02 02:35:02,2020-01-02 02:40:29
4,2020-01-02 00:00:00,0,1,0.0,4,0,31.32,4,1.000000,1.000000,...,N,N,N,N,湖南,2020-01-02 09:13:33,2020-01-05 08:57:56,2020-01-02 09:12:18,2020-01-02 09:10:49,2020-01-02 09:11:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21312,2020-01-22 00:00:00,0,2,0.0,30,0,3552.00,30,92.000000,92.000000,...,N,N,N,N,湖北,2020-01-23 09:25:18,2020-04-01 13:23:25,2020-01-22 13:20:40,2020-01-22 13:06:14,2020-01-22 13:08:39
21313,2020-01-25 00:00:00,0,1,0.0,242,0,670.00,242,0.122000,0.202000,...,N,N,N,N,湖北,2020-01-29 10:25:48,2020-01-29 10:37:39,2020-01-28 16:59:00,2020-01-25 10:55:18,2020-01-28 15:28:04
21314,2020-01-28 00:00:00,0,1,0.0,1060,0,18780.00,1060,0.245625,0.245625,...,N,N,N,N,湖北,2020-01-29 10:25:52,2020-01-29 10:37:39,2020-01-28 16:59:13,2020-01-28 13:42:47,2020-01-28 15:23:21
21315,2020-01-30 00:00:00,0,2,0.0,40,0,365.00,40,0.033333,0.033333,...,N,N,N,N,湖北,2020-01-30 13:56:05,2020-01-30 14:07:12,2020-01-30 11:13:11,2020-01-30 11:08:46,2020-01-30 11:13:11
